In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    my first published notebook
    if you find an error, write

        based on:

1. http://www.kaggle.com/its7171/mmdetection-for-segmentation-training
1. https://www.kaggle.com/sreevishnudamodaran/vinbigdata-fusing-bboxes-coco-dataset
1. https://www.kaggle.com/anandsm7/hpa-starter-pytorch-eda-classification-nfnets

In [ ]:
!pip install ../input/mmcoco/mmpycocotools/

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
from PIL import Image
import shutil
import csv
import threading
from threading import Thread
import sqlite3
import datetime
from tqdm import tqdm
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pickle
import cv2
from multiprocessing import Pool
import matplotlib.pyplot as plt
import json
from multiprocessing import Process, Queue

In [ ]:
train = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
train.head()

In [ ]:
# создать столбец с количеством лейблов изображения
train['label_count'] = train['Label'].apply(lambda x: len(x.split("|")))
train.head()

In [ ]:
# количество одиночных лейблов
a = train[train['label_count'] == 1]['label_count'].count()
a

In [ ]:
#количество составных лейблов
b = train[train['label_count'] > 1]['label_count'].count()
b

In [ ]:
#now lets compare between single vs multi label distribution
# теперь давайте сравним распределение по одной и по нескольким меткам
labels = train["Label"].apply(lambda x: x.split("|"))
labels_count = defaultdict(int)

# Update the counter 
for label in labels:
    if len(label) > 1:
        for l in label:
            labels_count[LABELS[int(l)]]+=1
    else:
        labels_count[LABELS[int(label[0])]]+=1
        
plt.figure(figsize=(10, 8))
plt.xticks(rotation=45)
plt.title("Target counts")
sns.barplot(list(labels_count.keys()),list(labels_count.values()))
plt.show()

In [ ]:
train['Label'].value_counts()

In [ ]:
massive = []
for i in range(18):
    f = len(train[train['Label'] == f'{i}'])
    massive.append(f)
    print(f'{i} - {f}')
sum = 0

sum = 0
for i in massive:
    sum += int(i)
print(sum)

In [ ]:
d = len(train[train['Label'] == '11'])
d

In [ ]:
h = train.loc[train.Label == '11']
h

In [ ]:
# посмотреть изображение с лейблом 11
def show_image():
    sns.reset_orig()
    #get image id
    im_id = 'b6a469d8-bbad-11e8-b2ba-ac1f6b6435d0'

    cdict1 = {'red':   ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)),

             'green': ((0.0,  0.0, 0.0),
                       (0.75, 1.0, 1.0),
                       (1.0,  1.0, 1.0)),

             'blue':  ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0))}

    cdict2 = {'red':   ((0.0,  0.0, 0.0),
                       (0.75, 1.0, 1.0),
                       (1.0,  1.0, 1.0)),

             'green': ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)),

             'blue':  ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0))}

    cdict3 = {'red':   ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)),

             'green': ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0)),

             'blue':  ((0.0,  0.0, 0.0),
                       (0.75, 1.0, 1.0),
                       (1.0,  1.0, 1.0))}

    cdict4 = {'red': ((0.0,  0.0, 0.0),
                       (0.75, 1.0, 1.0),
                       (1.0,  1.0, 1.0)),

             'green': ((0.0,  0.0, 0.0),
                       (0.75, 1.0, 1.0),
                       (1.0,  1.0, 1.0)),

             'blue':  ((0.0,  0.0, 0.0),
                       (1.0,  0.0, 0.0))}

    plt.register_cmap(name='greens', data=cdict1)
    plt.register_cmap(name='reds', data=cdict2)
    plt.register_cmap(name='blues', data=cdict3)
    plt.register_cmap(name='yellows', data=cdict4)

    #get each image channel as a greyscale image (second argument 0 in imread)
    # получить каждый канал изображения как изображение в оттенках серого (второй аргумент 0 в imread)
    green = cv2.imread('../input/hpa-single-cell-image-classification/train/{}_green.png'.format(im_id), 0)
    red = cv2.imread('../input/hpa-single-cell-image-classification/train/{}_red.png'.format(im_id), 0)
    blue = cv2.imread('../input/hpa-single-cell-image-classification/train/{}_blue.png'.format(im_id), 0)
    yellow = cv2.imread('../input/hpa-single-cell-image-classification/train/{}_yellow.png'.format(im_id), 0)


    #display each channel separately
    # отображать каждый канал отдельно
    fig, ax = plt.subplots(nrows = 2, ncols=2, figsize=(15, 15))
    ax[0, 0].imshow(green, cmap="greens")
    ax[0, 0].set_title("Protein of interest", fontsize=18)
    ax[0, 1].imshow(red, cmap="reds")
    ax[0, 1].set_title("Microtubules", fontsize=18)
    ax[1, 0].imshow(blue, cmap="blues")
    ax[1, 0].set_title("Nucleus", fontsize=18)
    ax[1, 1].imshow(yellow, cmap="yellows")
    ax[1, 1].set_title("Endoplasmic reticulum", fontsize=18)
    for i in range(2):
        for j in range(2):
            ax[i, j].set_xticklabels([])
            ax[i, j].set_yticklabels([])
            ax[i, j].tick_params(left=False, bottom=False)
    plt.show()
    

show_image()

    Create a selection from images with 1 label
    Создаем выборку из изображений с 1 меткой

In [ ]:
my_dict = {}
with open("../input/hpa-single-cell-image-classification/train.csv","r") as f:
    cr = csv.reader(f, delimiter=',', lineterminator='\n')
    next(f, None)
    for row in cr:
        if len(row[1]) <= 2 and int(row[1]) < 18:
            a = row[1].strip('", ')
            my_dict.setdefault(row[0].strip('" '),[]).append(a)
with open("./train_1.csv", "w") as f:
    writer = csv.writer(f,delimiter=',')
    writer.writerow(('ID', 'Label'))
    for i,j in my_dict.items():
        d=" "
        t = d.join(j)
        writer.writerow([i]+[t])

In [ ]:
con = sqlite3.connect('./bas.db')
cursorObj = con.cursor()
cursorObj.execute('CREATE TABLE IF NOT EXISTS tabel(id INTEGER PRIMARY KEY AUTOINCREMENT, IDS REAL not NULL, LABEL REAL not NULL)')
con.commit()
with open("./train_1.csv","r") as fg:
    cr = csv.reader(fg, delimiter=',', lineterminator='\n')
    next(fg, None)
    for a, b in cr:
        cursorObj.execute("INSERT INTO tabel(IDS, LABEL) VALUES (?,?)", (a, b))  # Добавляем без id - он добавится сам
con.commit()
cursorObj.close()
con.close()

In [ ]:
con = sqlite3.connect('./bas.db')
cursorObj = con.cursor()
datas = {}
for i in range(18):
    cursorObj.execute(f"SELECT DISTINCT id, IDS, LABEL FROM tabel WHERE LABEL= '{i}' LIMIT 200")
    rows = cursorObj.fetchall()
    for a, b, c in rows[0:180]:
        if c == 0.0:
            v = '18'
            d = '1'
            datas.setdefault(b.strip('" '),[]).append([v, d])
        else:   
            d = '1'
            datas.setdefault(b.strip('" '),[]).append([c, d])
    for f, g, h in rows[181:200]:
        if h == 0.0:
            y = '18'
            i = '2'
            datas.setdefault(g.strip('" '),[]).append([y, i])
        else:
            i = '2'
            datas.setdefault(g.strip('" '),[]).append([h, i])
con.commit()
cursorObj.close()
con.close() 

with open("./train_2.csv", "w") as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(('id', 'label', 'val_tr'))
    for k,l in datas.items():
        q = int(l[0][0])
        r = l[0][1]
        writer.writerow([k]+[q]+[r])

    Selecting images for training
    Отбираем изображения для тренировки

In [ ]:
!mkdir ./otbor

In [ ]:
paths = pd.read_csv('./train_2.csv')
files = '../input/hpa-single-cell-image-classification/train/'
outp = './otbor/'
imagepaths = paths['id']
imagepaths.head()

In [ ]:
for i, path in tqdm(enumerate(imagepaths)):
    img_a  = f'{files}{path}_red.png'
    img_b  = f'{files}{path}_green.png'
    img_c  = f'{files}{path}_blue.png'
    img_d  = f'{files}{path}_yellow.png'
    shutil.copy2(img_a, outp)
    shutil.copy2(img_b, outp)
    shutil.copy2(img_c, outp)
    shutil.copy2(img_d, outp)

In [ ]:
# for RGB images
# для RGB images
for i, path in tqdm(enumerate(imagepaths)):
    img_a  = f'{files}{path}.jpg'
    shutil.copy2(img_a, outp)

In [ ]:
paths_m = pd.read_csv('../input/train-rgb-200/train_2.csv')
files_m = '../input/train-rgb-200/xotb_200_m'
outp_m = './otbor_m/'
imagepaths = paths_m['id']
imagepaths.head()

In [ ]:
for i, path in tqdm(enumerate(imagepaths)):
    img_m  = f'{files_m}{path}.npz'
    shutil.copy2(img_m, outp_m)

In [ ]:
%%bash
cd ./
zip -r ./otbor_m ./*

In [ ]:
from IPython.display import FileLink
FileLink(r'./otbor_m.zip')

    Create annotation files
    Создаем файлы аннотаций

In [ ]:
cell_dir = '../input/otbor200/xotb_200_m/'
im_dir = '../input/otbor200/xotb_200/'
tr = pd.read_csv('./train_2.csv')
#out_dir = './otbor_m'

In [ ]:
tr.head()

In [ ]:
tr['label'].value_counts()

In [ ]:
tr.loc[tr['id']=='b6a469d8-bbad-11e8-b2ba-ac1f6b6435d0']

In [ ]:
tr_set = tr.loc[tr['val_tr']== 1]
tr_set.head()

In [ ]:
tr_set.label.value_counts()

In [ ]:
val_set = tr.loc[tr['val_tr']== 2]

In [ ]:
val_set['label'].value_counts()

In [ ]:
# Class 0 has been moved to number 18 (important). remember to return it to number 0 after checking the model
# Класс 0 был перемещен на  номер 18 (важно). не забыть вернуть его на номер 0 после проверки модели
labels = {
0: "back",
1: "Nuclear membrane",
2: "Nucleoli",
3: "Nucleoli fibrillar center",
4: "Nuclear speckles",
5: "Nuclear bodies",
6: "Endoplasmic reticulum",
7: "Golgi apparatus",
8: "Intermediate filaments",
9: "Actin filaments",
10: "Microtubules",
11: "Mitotic spindle",
12: "Centrosome",
13: "Plasma membrane",
14: "Mitochondria",
15: "Aggresome",
16: "Cytosol",
17: "Vesicles and punctate cytosolic patterns",
18: "Nucleoplasm"
}

In [ ]:
now = datetime.datetime.now()

data = dict(
    images=[
        # license, url, file_name, height, width, date_captured, id
    ],
    type='instances',
    annotations=[
        # segmentation, area, iscrowd, image_id, bbox, category_id, id
    ],
    categories=[
        # supercategory, id, name
    ],
)

In [ ]:
class_name_to_id = {}
for i, each_label in enumerate(labels):
    class_name = each_label
    class_id = i
    class_name_to_id[class_name] = class_id
    data['categories'].append(dict(
        supercategory=None,
        id=class_id,
        name=str(class_name),
    ))

In [ ]:
data

In [ ]:
# Определены следующие функции API:
# encode - Кодировать двоичные маски с помощью RLE.
# decode - декодировать двоичные маски, закодированные с помощью RLE.
# merge - вычислить объединение или пересечение закодированных масок.
# iou - вычислить пересечение по объединению масок.
# area - Расчетная область закодированных масок.
# toBbox - получить ограничивающие рамки, окружающие закодированные маски.
# frPyObjects - Преобразование многоугольника, bbox и несжатого RLE в закодированную маску RLE.

#  Rs     = encode( masks )
#  masks  = decode( Rs )
#  R      = merge( Rs, intersect=false )
#  o      = iou( dt, gt, iscrowd )
#  a      = area( Rs )
#  bbs    = toBbox( Rs )
#  Rs     = frPyObjects( [pyObjects], h, w )

# In the API the following formats are used:
#  Rs      - [dict] Run-length encoding of binary masks
#  R       - dict Run-length encoding of binary mask
#  masks   - [hxwxn] Binary mask(s) (must have type np.ndarray(dtype=uint8) in column-major order)
#  iscrowd - [nx1] list of np.ndarray. 1 indicates corresponding gt image has crowd region to ignore
#  bbs     - [nx4] Bounding box(es) stored as [x y w h]
#  poly    - Polygon stored as [[x1 y1 x2 y2...],[x1 y1 ...],...] (2D list)
#  dt,gt   - May be either bounding boxes or encoded masks
# Both poly and bbs are 0-indexed (bbox=[0 0 1 1] encloses first pixel).

In [ ]:
train_out_file = './train_annotations.json'
val_out_file = './val_annotations.json'
data_val = data.copy()
data_val['images'] = []
data_val['annotations'] = []
data_train = data.copy()
data_train['images'] = []
data_train['annotations'] = []

In [ ]:
# print utility from public notebook
# утилита печати из общедоступной записной книжки
def print_masked_img(image_id, mask):
    img = load_RGBY_image(image_id, im_dir)
    
    plt.figure(figsize=(35, 35))
    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask)
    plt.title('Mask')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(img)
    plt.imshow(mask, alpha=0.6)
    plt.title('Image + Mask')
    plt.axis('off')
    plt.show()
    
# image loader, using rgb only here
# загрузчик изображений, здесь используется только rgb
def load_RGBY_image(image_id, train_or_test = im_dir, image_size=None):
    stacked_images = read_img(image_id, train_or_test, image_size)
    #red = read_img(image_id, "red", train_or_test, image_size)
    #green = read_img(image_id, "green", train_or_test, image_size)
    #blue = read_img(image_id, "blue", train_or_test, image_size)
    #yellow = read_img(image_id, "yellow", train_or_test, image_size)
    #stacked_images = np.transpose(np.array([red, green, blue]), (1,2,0))
    return stacked_images

# 
# def read_img(image_id, color, train_or_test = im_dir, image_size=None):
def read_img(image_id, im_dir, image_size=None):
    filename = f'{im_dir}/{image_id}.jpg'
    #filename = f'{im_dir}/{image_id}_{color}.png'
    assert os.path.exists(filename), f'not found {filename}'
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if image_size is not None:
        img = cv2.resize(img, (image_size, image_size))
    if img.max() > 255:
        img_max = img.max()
        img = (img/255).astype('uint8')
    return img

In [ ]:
image_id = '0051ccbc-bbbb-11e8-b2ba-ac1f6b6435d0'
cell_mask = np.load('../input/otbor200/xotb_200_m/0051ccbc-bbbb-11e8-b2ba-ac1f6b6435d0.npz')['arr_0']
print_masked_img(image_id, cell_mask)

In [ ]:
MAX_GREEN = 64 # filter out dark green cells # отфильтровать темно-зеленые клетки
def get_rles_from_mask(image_id, class_id):
    mask = np.load(f'{cell_dir}/{image_id}.npz')['arr_0']
    rle_list = []
    mask_ids = np.unique(mask)
    for val in mask_ids:
        #print(val)
        if val == 0:
            continue
        binary_mask = np.where(mask == val, 1, 0).astype(np.uint8)
        rle = coco_rle_encode(binary_mask)
        rle_list.append(rle)
    return rle_list, mask.shape[0], mask.shape[1]

def coco_rle_encode(mask):
    rle = {'counts': [], 'size': list(mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

# make annotation helper called multi processes
# делаем помощник по аннотации, называемый несколькими процессами
def mk_ann(idx):
    image_id = tr_set.iloc[idx].id
    class_id = tr_set.iloc[idx].label
    anno = coco_json(image_id, class_id)
    return anno, idx, image_id

In [ ]:
# convert segmentation mask image to run length encoding
# преобразовать изображение маски сегментации в кодировку длины прогона
# this part would take several DAYS, depends on your CPU power. below is an easier and faster way
# эта часть займет несколько ДНЕЙ, в зависимости от мощности вашего процессора. ниже более простой и быстрый способ
def coco_json(image_id, class_id):
    #for idx in range(len(tr_set)):
#         image_id = tr_set.iloc[idx].ID
#         class_id = tr_set.iloc[idx].Label
    rles, height, width = get_rles_from_mask(image_id, class_id)
    if len(rles) == 0:
        ## Add Images to annotation
        data_train['images'].append(dict(
            file_name = image_id+'.jpg',
            width = width,
            height = height,
            date_captured=None,
            id=idx))
        data_train['annotations'].append(dict())
    else:
        data_train['images'].append(dict(
                file_name = image_id+'.jpg',
                width = width,
                height = height,
                date_captured=None,
                id=idx))
        rles = mutils.frPyObjects(rles, height, width)
        bboxes = mutils.toBbox(rles)
    #     bboxes[:, 2] += bboxes[:, 0]
    #     bboxes[:, 3] += bboxes[:, 1]
        for bb, rl in zip(bboxes, rles):
            w = bb[2]
            h = bb[3]
            bbox =[
                    bb[0],
                    bb[1],
                    w,
                    h]
            area = (w)*(h)
            data_train['annotations'].append(dict(id=len(data_train['annotations']),
                                                  area=area, 
                                                  bbox=bbox,
                                                  iscrowd= 0, #1,
                                                  image_id=idx,
                                                  category_id=int(class_id)))
                                                  #segmentation = rl ))
        return data_train


#     with open(train_out_file, 'w') as f:
#         json.dump(data_train, f, indent=4)

In [ ]:
# this part would take several hours, depends on your CPU power.
MAX_THRE = 4 # set your avarable CPU count.
p = Pool(processes=MAX_THRE)
annos = []
len_df = len(tr_set)
for anno, idx, image_id in p.imap(mk_ann, range(len(tr_set))):
    if len(anno['annotations']) > 0:
        annos.append(anno)

In [ ]:
data_train

In [ ]:
with open(f'./custom_trn1.pkl', 'wb') as f:
    pickle.dump(data_train, f)

In [ ]:
with open(train_out_file, 'w') as f:
    json.dump(data_train, f, indent=4)

repeat the same with val_set

    pickle set
    faster way
    более быстрый способ

    replace val_set with tr_set to create train annotations
    заменить val_set на tr_set чтобы создать аннотации для train

In [ ]:
cell_dir = '../input/otbor200/xotb_200_m'

In [ ]:
# convert segmentation mask image to run length encoding
# преобразовать изображение маски сегментации в кодировку длины прогона
MAX_GREEN = 64 # filter out dark green cells # отфильтровать темно-зеленые клетки
def get_rles_from_mask(image_id, class_id):
    mask = np.load(f'{cell_dir}/{image_id}.npz')['arr_0']
    #if class_id != '18':
        #green_img = read_img(image_id, 'green')
    rle_list = []
    mask_ids = np.unique(mask)
    for val in mask_ids:
        if val == 0:
            continue
        binary_mask = np.where(mask == val, 1, 0).astype(bool)
        #if class_id != '18':
            #masked_img = green_img * binary_mask
            #print(val, green_img.max(),masked_img.max())
            #f masked_img.max() < MAX_GREEN:
                #continue
        rle = coco_rle_encode(binary_mask)
        rle_list.append(rle)
    return rle_list, mask.shape[0], mask.shape[1]

def coco_rle_encode(mask):
    rle = {'counts': [], 'size': list(mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

# mmdet custom dataset generator
# mmdet генератор пользовательских наборов данных
def mk_mmdet_custom_data(image_id, class_id):
    rles, height, width = get_rles_from_mask(image_id, class_id)
    if len(rles) == 0:
        return {
            'file_name': image_id+'.jpg',
            'width': width,
            'height': height,
            'ann': {}
        }
    rles = mutils.frPyObjects(rles, height, width)
    bboxes = mutils.toBbox(rles) 
    bboxes[:, 2] += bboxes[:, 0] # voc format
    bboxes[:, 3] += bboxes[:, 1] # voc format
    return {
        'file_name': image_id+'.jpg',
        'width': width,
        'height': height,
        'ann':
            {
                'bboxes': np.array(bboxes, dtype=np.float32),
                'labels': np.zeros(len(bboxes)), # dummy data.(will be replaced later) # фиктивные данные. (будут заменены позже)
                'masks': rles
            }
    }

In [ ]:
# make annotation helper called multi processes
# делаем помощник по аннотации, называемый несколькими процессами
def mk_ann(idx):
    image_id = val_set.iloc[idx].id
    class_id = val_set.iloc[idx].label
    anno = mk_mmdet_custom_data(image_id, class_id)
   # img = load_RGBY_image(image_id, im_dir)
   # cv2.imwrite(f'{out_dir}/{image_id}.jpg', img)
    return anno, idx, image_id

   Create annotations for training a model

In [ ]:
# this part would take several hours, depends on your CPU power.
MAX_THRE = 4 # set your avarable CPU count.
p = Pool(processes=MAX_THRE)
annos = []
len_df = len(val_set)
for anno, idx, image_id in p.imap(mk_ann, range(len(val_set))):
    if len(anno['ann']) > 0:
        annos.append(anno)
    print(f'{idx+1}/{len_df}, {image_id}')

In [ ]:
lbl_cnt_dict = val_set.set_index('id').to_dict()['label']
trn_annos = []
val_annos = []
#val_len = int(len(annos)*0.01)
for idx in range(len(annos)):
    ann = annos[idx]
    filename = ann['file_name'].replace('.jpg','').replace('.png','')
    #label_ids = lbl_cnt_dict[filename]
    label_id = lbl_cnt_dict[filename]
    len_ann = len(ann['ann']['bboxes'])
    bboxes = ann['ann']['bboxes']
    masks = ann['ann']['masks']
    # asign image level labels to each cells
    # присвоить метки уровня изображения каждой ячейке
    # print(label_ids)
    # for cnt, label_id in tqdm(enumerate(label_ids.split('|'))):
          #label_id = int(label_id)
#     if cnt == 0:
    ann['ann']['labels'] = np.full(len_ann, label_id)
#     else:
#     ann['ann']['bboxes'] = np.concatenate([ann['ann']['bboxes'],bboxes])
#     ann['ann']['labels'] = np.concatenate([ann['ann']['labels'],np.full(len_ann, label_id)])
#     ann['ann']['masks'] = ann['ann']['masks'] + masks    
#     if idx < val_len:
#         val_annos.append(ann)
#     else:
    trn_annos.append(ann)

    val_set change to tr_set 4 cells above to create train annotations
      val_set поменять на tr_set в 4 ячейках выше, чтобы создать аннотации для train

In [ ]:
with open(f'./coco_val.pkl', 'wb') as f:
    pickle.dump(trn_annos, f)

In [ ]:
with open(f'./coco_train.pkl', 'wb') as f:
    pickle.dump(trn_annos, f)

In [ ]:
sum = 0
for i in range(len(objects)):
    d = int(objects[i]['ann']['labels'][0])
    sum += 1
print(sum)

In [ ]:
# Read the annotations
file_path = "../input/pickles1/mmdet_val.pkl"
with open(file_path, 'rb') as f:
    data = pickle.load(f)
print(data[0])

In [ ]:
mask_list = data[0]['ann']['bboxes'][0]
#mask_list = mask_list.decode("utf-8")
#mask_list.dumps()
#mask_list.item(0)
mask_list.tolist()
#label = data[0]['ann']['labels'].tolist()
#label

In [ ]:
rls = data[0]['ann']['masks']
rls = mutils.decode(rls)
rls

In [ ]:
rls = data[0]['ann']['masks'][0]
rlsa = mutils.decode(rls) #.astype(bool)
rlsa
#a = rls[0].tolist()
#np.count_nonzero(a)

In [ ]:
for idx in range(5):
    image_id = data[0]['file_name'].replace('.jpg','').replace('.png','')
    rls = data[0]['ann']['masks'][idx]
    rlsa = mutils.decode(rls)
    print_masked_img(image_id, rlsa)

In [ ]:
color = (255,0,0)
imgs = []

image_id = data[0]['file_name'].replace('.jpg','').replace('.png','')
print(image_id)
rls = data[0]['ann']['bboxes'][2].tolist()
#x,y,w,h = rls
x = rls[0]
y = rls[1]
h = rls[2]
w = rls[3]
a = int(x)
b = int(y)
c = int(h)
d = int(w)
print(f'{x},{y},{h},{w}')
img = read_img(image_id, im_dir)
fr = cv2.rectangle(img,(x,y),(h,w),color,2)
window_name = 'image'
plt.figure(figsize=(25, 25))
plt.subplot(1, 3, 1)
plt.imshow(fr)
plt.title('Image')
plt.axis('off')

In [ ]:
train_set = pd.read_pickle('data/coco_trn1a.pkl')
valid_set = pd.read_pickle('data/coco__val_ful1a.pkl')
train_out_file = 'data/train_annotations.json'
val_out_file = 'data/val_annotations.json'
data_valid = data.copy()
data_valid['images'] = []
data_valid['annotations'] = []
data_train = data.copy()
data_train['images'] = []
data_train['annotations'] = []

In [ ]:
# преобразовать изображение маски сегментации в кодировку длины прогона
# this part would take several hours, depends on your CPU power.
# this part would take several hours, depends on your CPU power.
for idx, var in tqdm(enumerate(valid_set)):
    # print(f'{idx} - {var}')
#for idx in range(len(tr_set)):
    image_id = valid_set[idx]['file_name']
    class_id = valid_set[idx]['ann']['labels'][0]
    width = valid_set[idx]['width']
    height = valid_set[idx]['height']
    data_valid['images'].append(dict(
            file_name = image_id,
            width = width,
            height = height,
            date_captured=None,
            id=idx))
    bboxes = valid_set[idx]['ann']['bboxes'].astype(float)
    #rls = tr_set[idx]['ann']['masks']['counts']
    #rlsa = mutils.decode(rls)
    for i, bb in enumerate(bboxes):
        # print(f'{bb} - {i}')
    #for bb, rl in zip(bboxes, rles):
        x = float(bb[0])
        y = float(bb[1])
        w = float(bb[2])-x
        h = float(bb[3])-y
        bbox =[x, y, w, h]
        #bbox = np.array(bbox, dtype=np.float64)
        #print(bb)
        area = (w)*(h)
        #print(f'{bb[0]} {bb[1]} {w} {h} {area}')
        data_valid['annotations'].append(dict(id=len(data_valid['annotations']),
                                              area=area, 
                                              bbox=bbox,
                                              iscrowd= 0, #1,
                                              image_id=idx,
                                              category_id=int(class_id)))
                                              #segmentation = rl ))
        #return data_valid

In [ ]:
with open(val_out_file, 'w') as f:
    json.dump(data_valid, f, indent=4)

In [ ]:
# преобразовать изображение маски сегментации в кодировку длины прогона
# this part would take several hours, depends on your CPU power.
# this part would take several hours, depends on your CPU power.
for idx, var in tqdm(enumerate(train_set)):
    # print(f'{idx} - {var}')
#for idx in range(len(tr_set)):
    image_id = train_set[idx]['file_name']
    class_id = train_set[idx]['ann']['labels'][0]
    width = train_set[idx]['width']
    height = train_set[idx]['height']
    data_train['images'].append(dict(
            file_name = image_id,
            width = width,
            height = height,
            date_captured=None,
            id=idx))
    bboxes = train_set[idx]['ann']['bboxes'].astype(float)
    #rls = tr_set[idx]['ann']['masks']['counts']
    #rlsa = mutils.decode(rls)
    for i, bb in enumerate(bboxes):
        # print(f'{bb} - {i}')
    #for bb, rl in zip(bboxes, rles):
        x = float(bb[0])
        y = float(bb[1])
        w = float(bb[2])-x
        h = float(bb[3])-y
        bbox =[x, y, w, h]
        #bbox = np.array(bbox, dtype=np.float64)
        #print(bb)
        area = (w)*(h)
        #print(f'{bb[0]} {bb[1]} {w} {h} {area}')
        data_train['annotations'].append(dict(id=len(data_train['annotations']),
                                              area=area, 
                                              bbox=bbox,
                                              iscrowd= 0, #1,
                                              image_id=idx,
                                              category_id=int(class_id)))
                                              #segmentation = rl ))
        #return data_train

In [ ]:
with open(train_out_file, 'w') as f:
    json.dump(data_train, f, indent=4)